## Libraries

In [1]:
!pip install numpy
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install skll

import matplotlib.pyplot as plt

import sklearn.metrics as sklm
import skll.metrics as skllm
import copy
import json

import pandas as pd
import numpy as np

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Utils

In [2]:
PATH = "./"
ROOT_PATH = "../../"

In [3]:
def retrieve_data(flow_interval):
    path = "{0}dataset/dataset_flow_{1}.csv".format(ROOT_PATH, flow_interval)
    print(PATH)
    data = pd.read_csv(path, ';')
    
    data['Flow'].apply(int)
    data['AveSpeed'].apply(float)
    data['Density'].apply(float)
    data['Sunday'].apply(int)
    data['Monday'].apply(int)
    data['Tuesday'].apply(int)
    data['Wednesday'].apply(int)
    data['Thursday'].apply(int)
    data['Friday'].apply(int)
    data['Saturday'].apply(int)
      
    return data

In [4]:
def print_json (obj):
  print(json.dumps(obj, sort_keys=True, indent=4))

In [5]:
def store(obj, path, name):
  with open("{0}{1}/{2}.json".format(PATH, path, name), 'w') as json_file:
    json.dump(obj, json_file, sort_keys=True, indent=4)

## Testing

In [6]:
def plot_prediction (Y, Y_hat, title, alpha, gamma):
  """ Plot Prediction
  
  Plot the prediction (Flow x Time) of what was expected and what
  was predicted.
  """
  alpha_dec = str(alpha - int(alpha)).split('.')[1]
  gamma_dec = str(gamma - int(gamma)).split('.')[1]
  name = f"{title}_alpha_d{alpha_dec}_gamma_d{gamma_dec})"
  path = f"plots/prediction/{name}"
    
  plt.plot(Y)
  plt.plot(Y_hat)
  plt.title(f"Predição do Modelo {title} (alpha {alpha}, gamma {gamma})")
  plt.ylabel('Fluxo')
  plt.xlabel('Tempo')
  plt.legend(['esperado', 'observado'], loc='upper left')
  plt.rcdefaults()

  plt.savefig(path + ".png", bbox_inches='tight')
  # plt.savefig(path + ".pdf")

  plt.close('all')

In [7]:
# Model Parameters
FLOW_INTERVAL = 150 # the interval size for each flow

# from 0.0 to 1.0 in intervals of 0.01
ALPHAS = [x/100 for x in range(100)] 
GAMMAS = [x/100 for x in range(100)]

In [8]:
def double_exponential_smoothing(X, alpha, gamma):

  result_data = {
    'results': {},
    'meta': {
      'FLOW_INTERVAL': FLOW_INTERVAL,
    }
  }
  
  name = "DES"
  expected, observed = [], []
  W = 1

  X_test = np.array_split(X, 10)
  F = X_test[8][-1]
  Lm1 = F
  bm1 = 0
  for x in X_test[9]:
    L = (alpha * x) + ( (1 - alpha) * (Lm1 + bm1) )
    b = ( gamma * (L - Lm1) ) + ( (1 - gamma) * bm1 )
    F1 = L + (b*W)
    
    expected.append(int(x))
    observed.append(float(F))
    F = F1
    Lm1 = L
    bm1 = b
  
  result_data['results']['RMSE'] = sklm.mean_squared_error(expected, observed, squared=False)
  result_data['results']['MAE'] = sklm.mean_absolute_error(expected, observed)
  result_data['results']['Kappa'] = skllm.kappa(expected, observed)
  result_data['results']['expected'] = expected
  result_data['results']['observed'] = observed
  alpha_dec = str(alpha - int(alpha)).split('.')[1]
  gamma_dec = str(gamma - int(gamma)).split('.')[1]
  #store(result_data, "results", f"{name}_alpha_d{alpha_dec}_gamma_d{gamma_dec}")
  return result_data

In [9]:
global current_best
global heatmap_rmse
global heatmap_mae
global heatmap_kappa

data = retrieve_data(FLOW_INTERVAL)
sequence = np.array(data['Flow'])

heatmap_rmse = [[0 for j in range(100)] for i in range(100)]
heatmap_mae = [[0 for j in range(100)] for i in range(100)]
heatmap_kappa = [[0 for j in range(100)] for i in range(100)]

progress = -1
int_alpha = 0
int_gamma = 0
for alpha, gamma in [(x,y) for x in ALPHAS for y in GAMMAS]:
  
  result_data = double_exponential_smoothing(sequence, alpha, gamma)

  heatmap_rmse[int_alpha][int_gamma] = result_data["results"]["RMSE"]
  heatmap_mae[int_alpha][int_gamma] = result_data["results"]["MAE"]
  heatmap_kappa[int_alpha][int_gamma] = result_data["results"]["Kappa"]

  int_gamma += 1

  if progress < alpha:
    if progress != -1:
      int_alpha += 1
    int_gamma = 0
    print(f'\rProgress: {int(alpha*100) + 1}%', end="")
    progress = alpha
  
  #expected = result_data['results']['expected']
  #observed = result_data['results']['observed']
  #plot_prediction(expected, observed, "DES", alpha, gamma)

./
Progress: 100%

# Plot 

In [10]:
!pip install plotly
import plotly.express as px

Defaulting to user installation because normal site-packages is not writeable


In [11]:
fig = px.imshow(heatmap_rmse, labels=dict(x="Alpha", y="Gamma", color="RMSE"), x = ALPHAS, y = GAMMAS)
fig.show()

In [12]:
fig = px.imshow(heatmap_mae, labels=dict(x="Alpha", y="Gamma", color="MAE"), x = ALPHAS, y = GAMMAS)
fig.show()

In [13]:
flat_list = [item for sublist in heatmap_mae for item in sublist]

In [14]:
min(flat_list)

0

In [15]:
fig = px.imshow(heatmap_kappa, labels=dict(x="Gamma", y="Alpha", color="Kappa"), x = GAMMAS, y = ALPHAS)
fig.show()

In [16]:
flat_list = [item for sublist in heatmap_kappa for item in sublist]

In [17]:
max(flat_list)

0.1661798523654313